In [42]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession

In [43]:
try:
    sc.stop()
except:
    pass
spark = SparkSession \
    .builder.enableHiveSupport() \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [44]:
df = spark.read.json("/Users/tq/Downloads/logics_shard1009.07.03.2017.log")

In [45]:
df.createOrReplaceTempView('RawTable')
preparetable = 'RawTable'

In [46]:
result1 = spark.sql(u'''
SELECT
    *
FROM %s
WHERE accountid is not null 
AND type_name!="CorpLevelChg" 
'''%preparetable).cache().createOrReplaceTempView('jsonTable')

In [47]:
spark.sql("select logtime from jsonTable limit 1").show()

+-------------------+
|            logtime|
+-------------------+
|1488844668542401853|
+-------------------+



In [49]:
# SELECT * FROM RawTable WHERE accountid is not null AND dt >= 20160720


result2 = spark.sql(u"""
SELECT
    t.acid AS acid,
    SUBSTRING(t.lt,0,10) AS lt,
    CAST(jsonTable.utc8 as DATE) AS date,
    jsonTable.sid AS sid,
    jsonTable.gid AS gid,
    jsonTable.channel AS ch,
    jsonTable.info.MachineType AS machine
FROM (
    SELECT
        accountid AS acid,
        min(logtime) AS lt
    FROM jsonTable
    WHERE type_name="Login"
    GROUP BY accountid
) t
LEFT JOIN jsonTable ON jsonTable.accountid = t.acid AND jsonTable.logtime = t.lt
""").cache().createOrReplaceTempView('accountID_ch')

In [52]:
result = spark.sql(u"""
SELECT * 
FROM accountID_ch
""")

In [53]:
result.show()

+--------------------+----------+----------+----+---+------------+--------------------+
|                acid|        lt|      date| sid|gid|          ch|             machine|
+--------------------+----------+----------+----+---+------------+--------------------+
|200:1009:b43ad44c...|1488929570|2017-03-08|1009|200|130134004600|     HUAWEI FRD-AL00|
|200:1009:d47fd3d4...|1488879408|2017-03-07|1009|200|130134004600|           Meizu M3s|
|200:1009:6a114571...|1488846543|2017-03-07|1009|200|        5000|    samsung SM-A7009|
|200:1009:d62581a9...|1488882221|2017-03-07|1009|200|130134005400|       Xiaomi MI MAX|
|200:1009:fff96d0d...|1488907118|2017-03-08|1009|200|         216|         Xiaomi MI 5|
|200:1009:e33321ce...|1488903137|2017-03-08|1009|200|130134001300|       OPPO OPPO R7s|
|200:1009:3a0685ed...|1488894721|2017-03-07|1009|200|        5000|    samsung SM-N9100|
|200:1009:31163d3e...|1488889152|2017-03-07|1009|200|130134001300|HUAWEI HUAWEI NXT...|
|200:1009:2b5d190e...|1488858738

In [149]:
x = result.replace(["5000","130134000001"],["0","1"],"ch")

In [150]:
# x.filter("uid= 'ee98bba8-9e9b-4db2-93aa-823127b924c1'").collect()

In [154]:
x.show()

+--------------------+----------+----+---+------------+--------------------+----------+
|                 uid|      date| sid|gid|          ch|             machine|        lt|
+--------------------+----------+----+---+------------+--------------------+----------+
|ee98bba8-9e9b-4db...|2017-03-07|1009|200|           1|        Meizu PRO 6s|1488845851|
|8858ad42-87ac-438...|2017-03-07|1009|200|130134000201|         Xiaomi MI 2|1488898373|
|4d866c11-5fbc-474...|2017-03-07|1009|200|           0|    samsung SM-A9100|1488845358|
|caa0b5f0-790b-406...|2017-03-07|1009|200|130134001300| Xiaomi Redmi Note 4|1488848262|
|f130ba9e-dab3-429...|2017-03-07|1009|200|130134001500|       Meizu m2 note|1488855421|
|8e551efd-ede3-4b1...|2017-03-07|1009|200|130134001335|    LeMobile Le X620|1488849044|
|adf1fa44-9ec2-452...|2017-03-07|1009|200|130134005600|华为p8兰博基尼限量版 HUAWE...|1488887172|
|26f5011d-5f61-476...|2017-03-07|1009|200|130134004600|    vivo vivo X7Plus|1488869543|
|751dde93-019b-4ec...|2017-03-07

In [155]:
properties = {
    "user": "test1",
    "password": "QmPhaQ8hYsxx"
}

In [156]:
x.write.jdbc("jdbc:mysql://54.223.192.252:3306/test", "bigdatatest22","append", properties)